In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [5]:
DATA_PATH = "/content/drive/MyDrive/LoL dataset/"
SEED = 42

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device

In [6]:
test = pd.read_csv(f'{DATA_PATH}test.csv')
train = pd.read_csv(f'{DATA_PATH}train.csv')

<ipython-input-6-9301b75e28d7>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(f'{DATA_PATH}test.csv')


In [ ]:
test

,gameid,league,year,split,playoffs,date,game,patch,participantid,side,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,LOLTMNT02_66852,LCK,2024,Spring,1,2024-03-30 06:10:37,1,14.06,1,Blue,...,220.0,-1694.0,-764.0,-25.0,1.0,5.0,3.0,4.0,3.0,1.0
1,LOLTMNT02_66852,LCK,2024,Spring,1,2024-03-30 06:10:37,1,14.06,2,Blue,...,181.0,-1021.0,-842.0,-26.0,1.0,5.0,1.0,1.0,7.0,0.0
2,LOLTMNT02_66852,LCK,2024,Spring,1,2024-03-30 06:10:37,1,14.06,3,Blue,...,230.0,1351.0,155.0,18.0,3.0,3.0,1.0,1.0,5.0,2.0
3,LOLTMNT02_66852,LCK,2024,Spring,1,2024-03-30 06:10:37,1,14.06,4,Blue,...,240.0,600.0,-100.0,13.0,2.0,5.0,0.0,2.0,5.0,1.0
4,LOLTMNT02_66852,LCK,2024,Spring,1,2024-03-30 06:10:37,1,14.06,5,Blue,...,28.0,-98.0,1357.0,15.0,0.0,7.0,3.0,0.0,6.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67375,LOLTMNT02_194401,KeSPA,2025,NaN,0,2024-12-08 09:03:13,4,14.23,8,Red,...,211.0,-1050.0,1845.0,36.0,0.0,2.0,2.0,4.0,1.0,2.0
67376,LOLTMNT02_194401,KeSPA,2025,NaN,0,2024-12-08 09:03:13,4,14.23,9,Red,...,253.0,-827.0,-702.0,-3.0,1.0,0.0,0.0,1.0,5.0,0.0
67377,LOLTMNT02_194401,KeSPA,2025,NaN,0,2024-12-08 09:03:13,4,14.23,10,Red,...,35.0,-146.0,-383.0,-5.0,0.0,4.0,2.0,0.0,5.0,2.0
67378,LOLTMNT02_194401,KeSPA,2025,NaN,0,2024-12-08 09:03:13,4,14.23,100,Blue,...,847.0,3278.0,3672.0,85.0,8.0,16.0,7.0,7.0,9.0,8.0


In [7]:
remove_list = [ 'gameid', 'split', 'game', 'participantid',
 'playername', 'teamname', 'champion', 'gamelength',
 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'doublekills',
 'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'firstbloodkill',
 'firstbloodassist', 'firstbloodvictim', 'team kpm', 'ckpm', 'firstdragon',
 'dragons', 'opp_dragons', 'elementaldrakes', 'opp_elementaldrakes',
 'infernals', 'mountains', 'clouds', 'oceans', 'chemtechs', 'hextechs',
 'dragons (type unknown)', 'elders', 'opp_elders', 'firstherald', 'heralds',
 'opp_heralds', 'void_grubs', 'opp_void_grubs', 'firstbaron', 'barons',
 'opp_barons', 'firsttower', 'towers', 'opp_towers', 'firstmidtower',
 'firsttothreetowers', 'turretplates', 'opp_turretplates', 'inhibitors',
 'opp_inhibitors', 'damagetochampions', 'dpm', 'damageshare',
 'damagetakenperminute', 'damagemitigatedperminute', 'wardsplaced', 'wpm',
 'wardskilled', 'wcpm', 'controlwardsbought', 'visionscore', 'vspm',
 'totalgold', 'earnedgold', 'earned gpm', 'earnedgoldshare', 'goldspent',
 'gspd', 'gpr', 'total cs', 'minionkills', 'monsterkills',
 'monsterkillsownjungle', 'monsterkillsenemyjungle', 'cspm', 'goldat10',
 'xpat10', 'csat10', 'opp_goldat10', 'opp_xpat10', 'opp_csat10',
 'golddiffat10', 'xpdiffat10', 'csdiffat10', 'killsat10', 'assistsat10',
 'deathsat10', 'opp_killsat10', 'opp_assistsat10', 'opp_deathsat10',
 'goldat15', 'xpat15', 'csat15', 'opp_goldat15', 'opp_xpat15', 'opp_csat15',
 'golddiffat15', 'xpdiffat15', 'csdiffat15', 'killsat15', 'assistsat15',
 'deathsat15', 'opp_killsat15', 'opp_assistsat15', 'opp_deathsat15',
 'goldat20', 'xpat20', 'csat20', 'opp_goldat20', 'opp_xpat20', 'opp_csat20',
 'golddiffat20', 'xpdiffat20', 'csdiffat20', 'killsat20', 'assistsat20',
 'deathsat20', 'opp_killsat20', 'opp_assistsat20', 'opp_deathsat20',
 'goldat25', 'xpat25', 'csat25', 'opp_goldat25', 'opp_xpat25', 'opp_csat25',
 'golddiffat25', 'xpdiffat25', 'csdiffat25', 'killsat25', 'assistsat25',
 'deathsat25', 'opp_killsat25', 'opp_assistsat25', 'opp_deathsat25']


In [8]:
test_ft  = test.drop(columns=remove_list)
train_ft  = train.drop(columns=remove_list)

In [9]:
test_ft.columns

Index(['league', 'year', 'playoffs', 'date', 'patch', 'side', 'position',
       'playerid', 'teamid', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1',
       'pick2', 'pick3', 'pick4', 'pick5', 'result'],
      dtype='object')

In [10]:
train = train_ft.drop('result', axis=1)
train_target = train_ft['result']

test = test_ft.drop('result', axis=1)
test_target = test_ft['result']

In [11]:
train_ft.head(3)

,league,year,playoffs,date,patch,side,position,playerid,teamid,ban1,ban2,ban3,ban4,ban5,pick1,pick2,pick3,pick4,pick5,result
0,LCKC,2022,0,2022-01-10 07:44:08,12.01,Blue,top,oe:player:38e0af7278d6769d0c81d7c4b47ac1e,oe:team:68911b3329146587617ab2973106e23,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,0
1,LCKC,2022,0,2022-01-10 07:44:08,12.01,Blue,jng,oe:player:637ed20b1e41be1c51bd1a4cb211357,oe:team:68911b3329146587617ab2973106e23,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,0
2,LCKC,2022,0,2022-01-10 07:44:08,12.01,Blue,mid,oe:player:d1ae0e2f9f3ac1e0e0cdcb86504ca77,oe:team:68911b3329146587617ab2973106e23,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,0


In [12]:
train_ft['month'] = pd.to_datetime(train_ft['date']).dt.month
train_ft['day'] = pd.to_datetime(train_ft['date']).dt.day
train_ft['time'] = pd.to_datetime(train_ft['date']).dt.time

train_ft = train_ft.drop(columns=['date'])

In [13]:
test_ft['month'] = pd.to_datetime(test_ft['date']).dt.month
test_ft['day'] = pd.to_datetime(test_ft['date']).dt.day
test_ft['time'] = pd.to_datetime(test_ft['date']).dt.time

test_ft = test_ft.drop(columns=['date'])

In [ ]:
train_ft

,league,year,patch,side,playerid,teamid,ban1,ban2,ban3,ban4,ban5,pick1,pick2,pick3,pick4,pick5,result,month,day,time
0,LCKC,2022,12.01,Blue,oe:player:38e0af7278d6769d0c81d7c4b47ac1e,oe:team:68911b3329146587617ab2973106e23,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,0,1,10,07:44:08
1,LCKC,2022,12.01,Blue,oe:player:637ed20b1e41be1c51bd1a4cb211357,oe:team:68911b3329146587617ab2973106e23,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,0,1,10,07:44:08
2,LCKC,2022,12.01,Blue,oe:player:d1ae0e2f9f3ac1e0e0cdcb86504ca77,oe:team:68911b3329146587617ab2973106e23,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,0,1,10,07:44:08
3,LCKC,2022,12.01,Blue,oe:player:998b3e49b01ecc41eacc392477a98cf,oe:team:68911b3329146587617ab2973106e23,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,0,1,10,07:44:08
4,LCKC,2022,12.01,Blue,oe:player:e9741b3a238723ea6380ef2113fae63,oe:team:68911b3329146587617ab2973106e23,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,0,1,10,07:44:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266035,PRM,2024,14.05,Red,oe:player:b6ebe3c8487ddc3a23c586e650c4528,oe:team:980ab072663574c2e4fca71f275d302,Orianna,Jinx,Sion,Ahri,Xin Zhao,NaN,NaN,NaN,NaN,NaN,1,4,12,19:00:53
266036,PRM,2024,14.05,Red,oe:player:9ea5afea85321b13df74acb62bfb530,oe:team:980ab072663574c2e4fca71f275d302,Orianna,Jinx,Sion,Ahri,Xin Zhao,NaN,NaN,NaN,NaN,NaN,1,4,12,19:00:53
266037,PRM,2024,14.05,Red,oe:player:a291e13e72870a901a7a03a59d9c6be,oe:team:980ab072663574c2e4fca71f275d302,Orianna,Jinx,Sion,Ahri,Xin Zhao,NaN,NaN,NaN,NaN,NaN,1,4,12,19:00:53
266038,PRM,2024,14.05,Blue,NaN,oe:team:aa8c3b781d96ddb7a24ab3240918988,Vi,Smolder,Volibear,Pyke,Poppy,Rumble,Zeri,Rell,Neeko,Viego,0,4,12,19:00:53


In [14]:
train_ft.columns

Index(['league', 'year', 'playoffs', 'patch', 'side', 'position', 'playerid',
       'teamid', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1', 'pick2',
       'pick3', 'pick4', 'pick5', 'result', 'month', 'day', 'time'],
      dtype='object')

In [15]:
train_ft.isnull().sum()

,0
league,0
year,0
playoffs,0
patch,0
side,0
position,0
playerid,47237
teamid,3264
ban1,792
ban2,438


In [18]:
columns_to_unknown = [
    'teamid',
    'ban1',
    'ban2',
    'ban3',
    'ban4',
    'ban5',
    'pick1',
    'pick2',
    'pick3',
    'pick4',
    'pick5',
    'position',
    'playerid',
    'playoffs']



In [19]:
for i in columns_to_unknown:
   train_ft[i] = train_ft[i].fillna('Unknown')

In [20]:
for i in columns_to_unknown:
   test_ft[i] = test_ft[i].fillna('Unknown')

In [21]:
train_ft['opp_teamid'] = train_ft.groupby("playerid")["teamid"].transform(lambda x: x.iloc[::-1].values)
train_ft['opp_playerid'] = train_ft.groupby("playerid")["playerid"].transform(lambda x: x.iloc[::-1].values)

In [22]:
test_ft['opp_teamid'] = test_ft.groupby("playerid")["teamid"].transform(lambda x: x.iloc[::-1].values)
test_ft['opp_playerid'] = test_ft.groupby("playerid")["playerid"].transform(lambda x: x.iloc[::-1].values)

In [23]:
test_ft.head(3)

,league,year,playoffs,patch,side,position,playerid,teamid,ban1,ban2,...,pick2,pick3,pick4,pick5,result,month,day,time,opp_teamid,opp_playerid
0,LCK,2024,1,14.06,Blue,top,oe:player:a1f3b2bb3f36e5113237f92d76dc4c8,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,Viego,Ashe,...,Unknown,Unknown,Unknown,Unknown,1,3,30,06:10:37,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,oe:player:a1f3b2bb3f36e5113237f92d76dc4c8
1,LCK,2024,1,14.06,Blue,jng,oe:player:dda4a873265dfd06df50a0d8a294908,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,Viego,Ashe,...,Unknown,Unknown,Unknown,Unknown,1,3,30,06:10:37,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,oe:player:dda4a873265dfd06df50a0d8a294908
2,LCK,2024,1,14.06,Blue,mid,oe:player:f9cd2ef9afea5cda0eb3263a025cec1,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,Viego,Ashe,...,Unknown,Unknown,Unknown,Unknown,1,3,30,06:10:37,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,oe:player:f9cd2ef9afea5cda0eb3263a025cec1


In [24]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

# train / test 쪼개야지....


In [27]:
# Train Features와 Target 저장
train.to_csv(f'{DATA_PATH}train_1226_p.csv', index=False)
train_target.to_csv(f'{DATA_PATH}train_target_1226_p.csv', index=False)

# Test Features와 Target 저장
test.to_csv(f'{DATA_PATH}test_1226_p.csv', index=False)
test_target.to_csv(f'{DATA_PATH}test_target_1226_p.csv', index=False)


- 인코딩

In [ ]:
train_ft.head(2)

,league,year,patch,side,playerid,teamid,ban1,ban2,ban3,ban4,...,pick2,pick3,pick4,pick5,result,month,day,time,opp_teamid,opp_playerid
0,LCKC,2022,12.01,Blue,oe:player:38e0af7278d6769d0c81d7c4b47ac1e,oe:team:68911b3329146587617ab2973106e23,Karma,Caitlyn,Syndra,Thresh,...,Unknown,Unknown,Unknown,Unknown,0,1,10,07:44:08,oe:team:28428f3022fc1ea52264b35912093dc,oe:player:38e0af7278d6769d0c81d7c4b47ac1e
1,LCKC,2022,12.01,Blue,oe:player:637ed20b1e41be1c51bd1a4cb211357,oe:team:68911b3329146587617ab2973106e23,Karma,Caitlyn,Syndra,Thresh,...,Unknown,Unknown,Unknown,Unknown,0,1,10,07:44:08,oe:team:20edec57649c30e1ed81e4b42634ac1,oe:player:637ed20b1e41be1c51bd1a4cb211357


In [ ]:
test_ft.head(2)

,league,year,patch,side,playerid,teamid,ban1,ban2,ban3,ban4,...,pick2,pick3,pick4,pick5,result,month,day,time,opp_teamid,opp_playerid
0,LCK,2024,14.06,Blue,oe:player:a1f3b2bb3f36e5113237f92d76dc4c8,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,Viego,Ashe,Varus,Jinx,...,Unknown,Unknown,Unknown,Unknown,1,3,30,06:10:37,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,oe:player:a1f3b2bb3f36e5113237f92d76dc4c8
1,LCK,2024,14.06,Blue,oe:player:dda4a873265dfd06df50a0d8a294908,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,Viego,Ashe,Varus,Jinx,...,Unknown,Unknown,Unknown,Unknown,1,3,30,06:10:37,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,oe:player:dda4a873265dfd06df50a0d8a294908


In [ ]:
test_target.head()

,result
0,1
1,1
2,1
3,1
4,1
